In [1]:
import keras
import tensorflow as tf
from keras.models import Model
from keras import Sequential, regularizers, preprocessing
from keras.layers import Input, Dense, Activation, Concatenate, BatchNormalization
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, UpSampling2D, Conv2D, MaxPooling2D, Add
import numpy as np
import os
import matplotlib.pyplot as plt
import numpy.random as rnd

In [2]:
def get_data(dir):
  images = {}
  masks = {}
  for file in os.listdir(dir + '/Images'):
    if(file != '.ipynb_checkpoints'):
      images[file] = keras.preprocessing.image.load_img(dir + '/Images/'+file, color_mode='grayscale')
  for file in os.listdir(dir + '/Masks'):
    if(file != '.ipynb_checkpoints'):
      masks[file] = keras.preprocessing.image.load_img(dir + '/Masks/'+file, color_mode='grayscale')

  return images, masks


def ResidualLayer(filters):
  def __Out(input):
    conv_1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(input)
    norm_1 = BatchNormalization()(conv_1)
    conv_2 = Conv2D(filters, (3, 3), padding='same', activation='relu')(norm_1)
    norm_2 = BatchNormalization()(conv_2)
    conv_skip = Conv2D(filters, (1, 1), padding='same')(input)
    result = Add()([norm_2, conv_skip])
    return Activation('relu')(result)
  return __Out

def generator(images, masks, batch_size):
  while True:
    batch_img = []
    batch_msk = []

    for i in range(batch_size):
      key = list(images)[np.random.randint(len(images.keys()))]
      lu_x = rnd.randint(images[key].size[0]-256)
      lu_y = rnd.randint(images[key].size[1]-256)
      crop_coord = [lu_x, lu_y, lu_x + 256, lu_y + 256]
      batch_img.append(np.array(images[key].crop(crop_coord)))
      batch_msk.append(np.array(masks[key].crop(crop_coord)))
      
    yield np.array(batch_img).reshape(batch_size, 256, 256, 1)/255, np.array(batch_msk).reshape(batch_size, 256, 256, 1)/255


def testdatagenerator(images, masks, batch_size):
  while True:
    batch_img = []
    batch_msk = []
    i = 0
    while i < batch_size:
      key = list(images)[np.random.randint(len(images.keys()))]
      lu_x = rnd.randint(images[key].size[0]-256)
      lu_y = rnd.randint(images[key].size[1]-256)
      crop_coord = [lu_x, lu_y, lu_x + 256, lu_y + 256]
      if(np.array(masks[key].crop(crop_coord)).sum()/255 > 10000):
        batch_img.append(np.array(images[key].crop(crop_coord)))
        batch_msk.append(np.array(masks[key].crop(crop_coord)))
        i+=1
      
    yield np.array(batch_img).reshape(batch_size, 256, 256, 1)/255, np.array(batch_msk).reshape(batch_size, 256, 256, 1)/255  


def Metric(threshold):
  IoU = keras.metrics.MeanIoU(2)
  def MetricIoU(y_true, y_pred):
    cond = tf.less(y_pred, tf.zeros(tf.shape(y_pred))+tf.constant(threshold))
    out = tf.where(cond, tf.zeros(tf.shape(y_pred)), tf.ones(tf.shape(y_pred)))
    return IoU(y_true, out)
  return MetricIoU

In [3]:
images, masks = get_data('./drive/MyDrive/Diplom/Train')
valid_images, valid_masks = get_data('./drive/MyDrive/Diplom/Valid')

In [4]:
basis = 4
drop = 0.1

inp = Input(shape=(None, None, 1))

resid_1 = ResidualLayer(basis)(inp)
pool_1 = MaxPooling2D(2)(resid_1)
drop_1 = Dropout(drop)(pool_1)

resid_2 = ResidualLayer(basis*2)(drop_1)
pool_2 = MaxPooling2D(2)(resid_2)
drop_2 = Dropout(drop)(pool_2)

resid_3 = ResidualLayer(basis*4)(drop_2)
pool_3 = MaxPooling2D(2)(resid_3)
drop_3 = Dropout(drop)(pool_3)

resid_4 = ResidualLayer(basis*8)(drop_3)
pool_4 = MaxPooling2D(2)(resid_4)
drop_4 = Dropout(drop)(pool_4)

resid_low = ResidualLayer(basis*16)(drop_4)

up_1 = UpSampling2D(2, interpolation='bilinear')(resid_low)
conc_1 = Concatenate()([resid_4, up_1])
drop_up_1 = Dropout(drop)(conc_1)

resid_up_1 = ResidualLayer(basis*8)(drop_up_1)
up_2 = UpSampling2D(2, interpolation='bilinear')(resid_up_1)
conc_2 = Concatenate()([resid_3, up_2])
drop_up_2 = Dropout(drop)(conc_2)

resid_up_2 = ResidualLayer(basis*4)(drop_up_2)
up_3 = UpSampling2D(2, interpolation='bilinear')(resid_up_2)
conc_3 = Concatenate()([resid_2, up_3])
drop_up_3 = Dropout(drop)(conc_3)

resid_up_3 = ResidualLayer(basis*2)(drop_up_3)
up_4 = UpSampling2D(2, interpolation='bilinear')(resid_up_3)
conc_4 = Concatenate()([resid_1, up_4])

resid_up_4 = ResidualLayer(basis)(conc_4)
conv_up_4 = Conv2D(1, (3, 3), padding='same')(resid_up_4)
result = Activation('sigmoid')(conv_up_4)


model = Model(inputs=inp, outputs=result)

In [5]:
model.compile('Adam', 'binary_crossentropy', metrics = Metric(0.3))

In [ ]:
batch_size = 32
history = model.fit_generator(generator(images, masks, batch_size),
                steps_per_epoch=100,
                epochs=100,
                verbose=1,
                validation_data = generator(valid_images, valid_masks, batch_size),
                validation_steps = 10)


In [ ]:
np.save('/content/drive/MyDrive/Diplom/History/history.npy',history.history)

In [ ]:
history=np.load('/content/drive/MyDrive/Diplom/History/history.npy',allow_pickle='TRUE').item()

In [ ]:
plt.figure()
plt.plot(hist1.history['val_MetricIoU'], label='Adam')
plt.plot(hist2.history['val_MetricIoU'], color='red', label='SGD')
plt.plot(hist3.history['val_MetricIoU'], color='green', label='Adagrad')
plt.plot(hist4.history['val_MetricIoU'], color='yellow', label='Rmsprop')
plt.legend()
plt.title('IOU')
plt.xlabel('epoch')

plt.figure()
plt.plot(hist1.history['val_loss'], label='Adam')
plt.plot(hist2.history['val_loss'], color='red', label='SGD')
plt.plot(hist3.history['val_loss'], color='green', label='Adagrad')
plt.plot(hist4.history['val_loss'], color='yellow', label='Rmsprop')
plt.title('Loss [75-100] epochs')
plt.xlabel('epoch')
plt.axis([75, 100, 0, 0.6])
plt.legend()

In [ ]:
model.save_weights(f'./drive/MyDrive/Diplom/Models/model.h5')

In [ ]:
x, y = generator(images, masks, 1).__next__()
plt.figure()
plt.imshow(model.predict(x).reshape(256, 256))
plt.figure()
plt.imshow(y.reshape(256, 256))


In [ ]:
valid_x, valid_y = generator(valid_images, valid_masks, 1).__next__()
plt.figure()
plt.imshow(model.predict(valid_x).reshape(256, 256)>(1/2), cmap = 'Greys_r')
plt.figure()
plt.imshow(valid_y.reshape(256, 256), cmap = 'Greys_r')
plt.figure()
plt.imshow(valid_x.reshape(256, 256), cmap = 'Greys_r')

In [ ]:
keras.utils.plot_model(model, show_shapes=False, show_layer_names=False, to_file='model.png')

In [8]:
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 933120000
BigImg = keras.preprocessing.image.load_img('./drive/MyDrive/Diplom/sq.png', color_mode='grayscale')

In [ ]:

import time 
array = np.zeros((BigImg.size[0], BigImg.size[1]))
shape = 256
for i in range(BigImg.size[0]//shape):
  print('\r', i/(BigImg.size[0]//shape), ' '*10, end='')
  for j in range(BigImg.size[1]//shape):
    crop_coord = [shape*i, shape*j, shape*i + shape, shape*j+ + shape]
    crop = BigImg.crop(crop_coord)
    npCrop = np.array(crop).reshape(1, shape, shape, 1)/255
    array[i*shape:(i+1)*shape,j*shape:(j+1)*shape] = np.array(model.predict(npCrop).reshape(shape, shape)).transpose()
threshold = 0.21 
array[array<threshold] = 0
array[array>threshold] = 1

In [ ]:
plt.imshow(array, cmap = 'Greys_r')

In [ ]:
PIL.Image.fromarray((array*255).astype('uint8'), mode='L').save('/content/drive/MyDrive/Diplom/sq_masked.png')